# Image Coordinate and Physical Coordinate
This notebooks discuss the indexing convension of the image coordinate and physical coordinate (defined in DS9) of DC2 dataset.

In [12]:
from lsst.daf.persistence import Butler
import lsst.geom as geom

## coadd

In [16]:
# get coadd Butler
coadd_repo = '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-coadd-wfd-dr6-v1-grizy'
coadd_butler = Butler(coadd_repo)
coadd_id = {'tract': 4639, 'patch': '1,1', 'filter': 'i'}
coadd_exp = coadd_butler.get('deepCoadd', coadd_id)
# coadd_exp.writeFits('coadd.fits')

CameraMapper INFO: Loading exposure registry from /global/cfs/cdirs/lsst/shared/DC2-prod/Run2.2i/desc_dm_drp/v19.0.0/registry.sqlite3
CameraMapper INFO: Loading calib registry from /global/cfs/cdirs/lsst/shared/DC2-prod/Run2.2i/desc_dm_drp/v19.0.0/CALIB/calibRegistry.sqlite3
CameraMapper INFO: Loading calib registry from /global/cfs/cdirs/lsst/shared/DC2-prod/Run2.2i/desc_dm_drp/v19.0.0/CALIB/calibRegistry.sqlite3
LsstCamMapper WARN: Unable to find valid calib root directory


The image coordinate and physical coordinate are not always the same for DC2 images. The image coordinate is indexed as 1, 2, ... in x and y direction. The physical coordinate, however, does not always start from 1. (e.g. Coadds with patch number deviates from '0,0' have physical coordinate starts about 4000)

The question is which coordinate is used by the stack functions.

We explore this question using a coadd exposure with patch '1,1'.

Below is the information from DS9.

|position                       |          image coordinate (x, y)      |        physical coordinate  (x, y)      |      sky (ra, dec)    |
|lower left corner              |                 (0, 0)                |                (3900, 3900)             |     (56.95, -31.05)   |
|image center                   |                 (2100, 2100)          |               (6000, 6000)              |     (56.82, -30.93)   |

In [8]:
# The beginX and beginY from bbox are the offsets
coadd_bbox = coadd_exp.getBBox()
coadd_begin_x = coadd_bbox.beginX
coadd_begin_y = coadd_bbox.beginY
print(coadd_begin_x, coadd_begin_y)

3900 3900


In [9]:
# It is unclear what is the pixel origin from the coadd_wcs
coadd_wcs = coadd_exp.getWcs()
print(coadd_wcs)

FITS standard SkyWcs:
Sky Origin: (56.3033175355, -30.4958677686)
Pixel Origin: (13999, 13999)
Pixel Scale: 0.2 arcsec/pixel


In [10]:
def radec2xy(ra, dec, wcs):
    radec = geom.SpherePoint(ra * geom.degrees, dec * geom.degrees)
    xy = geom.PointI(wcs.skyToPixel(radec))
    return xy.getX(), xy.getY()

In [15]:
# wcs.skyToPixel returns the physical coordinate, if we want to use the image coordinate, we need to subtract the offset.

In [13]:
radec2xy(56.95, -31.05, coadd_wcs)

(4026, 3995)

In [14]:
radec2xy(56.82, -30.93, coadd_wcs)

(6021, 6166)

## calexp

In [17]:
# get calexp Butler
calexp_repo = '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-calexp-v1'
calexp_butler = Butler(calexp_repo)
calexp_id = {'visit': 1013665, 'detector': 79, 'filter': 'i'}
calexp_exp = calexp_butler.get('calexp', calexp_id)
# calexp_exp.writeFits('calexp.fits')

CameraMapper INFO: Loading exposure registry from /global/cfs/cdirs/lsst/shared/DC2-prod/Run2.2i/desc_dm_drp/v19.0.0/registry.sqlite3
CameraMapper INFO: Loading calib registry from /global/cfs/cdirs/lsst/shared/DC2-prod/Run2.2i/desc_dm_drp/v19.0.0/CALIB/calibRegistry.sqlite3
CameraMapper INFO: Loading calib registry from /global/cfs/cdirs/lsst/shared/DC2-prod/Run2.2i/desc_dm_drp/v19.0.0/CALIB/calibRegistry.sqlite3


We use a calexp exposure for comparison

Information from DS9.
|Position                       |        image coordinate (x, y)        |        physical coordinate  (x, y)      |      sky (ra, dec)    |
|lower left corner              |                 (14.8, 21.6)          |                (14.8, 21.6)             |     (56.927, -31.057) |
|image center                   |                 (2047, 2204)          |               (2047, 2204)              |     (57.08, -31.16)   |

In [19]:
# No offset. 
calexp_bbox = calexp_exp.getBBox()
calexp_begin_x = calexp_bbox.beginX
calexp_begin_y = calexp_bbox.beginY
print(calexp_begin_x, calexp_begin_y)

0 0


In [20]:
# It seems that the pixel origin for calexp exposure represents the center.
calexp_wcs = calexp_exp.getWcs()
print(calexp_wcs)

FITS standard SkyWcs:
Sky Origin: (57.0765756225, -31.1436519190)
Pixel Origin: (2098.29, 1852.97)
Pixel Scale: 0.199748 arcsec/pixel


In [26]:
# wcs.skyToPixel returns the physical coordinate, which is the same as the image coordinate

In [24]:
radec2xy(56.927, -31.057, calexp_wcs)

(14, 4)

In [25]:
radec2xy(57.08, -31.16, calexp_wcs)

(2112, 2152)